# ETL Technical Document

Team: Richard Marshall, Ted Smiley, Joshua Gohlike

Introduction:
Our group created a database that allows analysts to explore the world of film and show the relationships between various data items that ultimately drive success at the box office.  These key data items included: film title, budget, imdb ratings, revenue, release date, Facebook likes, content rating, lead actor, Language, genre and length.

To do this, we followed the steps of the ETL process:

* **E**xtract
* **T**ransform
* **L**oad:  


Summary:
The dataset can now be used to answer questions like: does a high rating translate into higher revenue?  Do Facebook likes correlate with higher reviews?  What era of film has the highest rated films?  Which actors have the highest rated films?  Are those films also the highest rated? 


In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import datetime as dt

# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

* **Extract**: Our original data sources came from the IMDB 5000 Movie Dataset and the Movies Dataset (over 26M reviews of over 45,000 films).  We found the datasets on Kaggle and Dataworld, respectively.  Both datasets were formatted in CSV files. Once we downloaded the two CSV files, we were off and running.

In [2]:
# Read csv files into pandas dataframe
file1 = 'kaggle_movies_metadata.csv'
file2 = 'dataworld_movie_metadata.csv'

kaggle_movie_df = pd.read_csv(file1, dtype={'popularity': float})
dataworld_movie_df = pd.read_csv(file2)

* **Transform**: Before merging the CSV files, we needed to determine which columns were most significant and then drop the columns that were not needed.  This step was particularly important for the Movies Dataset from Kaggle because it was 24 columns wide and pared that down to seven columns that were of immediate interest to our group.  

In [3]:
# View df columns for editing
print(kaggle_movie_df.columns)
print(dataworld_movie_df.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')


In [4]:
# Drop unnecessary columns
k_df = kaggle_movie_df[['id', 'title', 'imdb_id', 'original_language', 'release_date', 'revenue', 'vote_average']]
d_df = dataworld_movie_df[['movie_title', 'gross', 'budget', 'genres', 'actor_1_name', 'language', 'country', 
                           'title_year', 'content_rating', 'imdb_score','movie_facebook_likes', 'movie_imdb_link']]

# Rename d_df column to use as key for joining dataframes
d_df = d_df.rename(columns={'movie_title': 'title'})

# Remove right most character from movie title in d_df
d_df['title'] = d_df['title'].str[:-1]

# Add id to dataworld dataframe by extracting from mivie_imdb_link
d_df['imdb_id'] = d_df['movie_imdb_link'].str[26:35]


# filter out unnecessary data: 1. non-english movies, 2. Release date is before 1960
k_df = k_df[k_df.original_language == 'en']
k_df = k_df[k_df.release_date > '12/31/1959']
d_df = d_df[d_df.language == 'English']
d_df = d_df[d_df.title_year > 1959]
d_df.head()

,title,gross,budget,genres,actor_1_name,language,country,title_year,content_rating,imdb_score,movie_facebook_likes,movie_imdb_link,imdb_id
0,Avatar,760505847.0,237000000.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,English,USA,2009.0,PG-13,7.9,33000,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,tt0499549
1,Pirates of the Caribbean: At World's End,309404152.0,300000000.0,Action|Adventure|Fantasy,Johnny Depp,English,USA,2007.0,PG-13,7.1,0,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,tt0449088
2,Spectre,200074175.0,245000000.0,Action|Adventure|Thriller,Christoph Waltz,English,UK,2015.0,PG-13,6.8,85000,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,tt2379713
3,The Dark Knight Rises,448130642.0,250000000.0,Action|Thriller,Tom Hardy,English,USA,2012.0,PG-13,8.5,164000,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,tt1345836
5,John Carter,73058679.0,263700000.0,Action|Adventure|Sci-Fi,Daryl Sabara,English,USA,2012.0,PG-13,6.6,24000,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,tt0401729


* **Load**:  We exported the two cleaned CSV files for import into a SQL database.  We chose this database type because we liked the usability and ubiquitous nature of SQL databases. The files were imported into a schema called "movies' and imported using the Table Data Import Wizard.

In [5]:
# Export cleaned tables to csv for load into mysql
k_df.to_csv("kaggle_movies_cleaned.csv")
d_df.to_csv("dataworld_movies_cleaned.csv")

In [6]:
# Create connection to sql server
server = 'cwru-data-project02.ciuevunbeloh.us-east-2.rds.amazonaws.com'
db = 'group02'
user = 'group02'
password = 'WquV9bNwwcdzZ8Wf'
engine_str = 'mysql+mysqlconnector://{}:{}@{}/{}'.format(user, password, server, db)
engine = create_engine(engine_str, echo=False, encoding='utf-8')
conn = engine.connect()

In [7]:
# Start a session
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [8]:
# Create database and import tables
k_df.to_sql('k_movies', conn, if_exists='replace')
d_df.to_sql('d_movies', conn, if_exists='replace')

* **Potential Analysis**: The following demonstrates a sampling of analyses that can be done with the cleaned datas, using pandas: Genre counts, Gross revenue by year, and rating averages by budget bins. A separate file containing sql queries allows for analysis in sql.

In [9]:
#Merge dataframes on movie title
df = pd.merge(k_df, d_df, on='imdb_id')

# View df
df

,id,title_x,imdb_id,original_language,release_date,revenue,vote_average,title_y,gross,budget,genres,actor_1_name,language,country,title_year,content_rating,imdb_score,movie_facebook_likes,movie_imdb_link
0,5,Four Rooms,tt0113101,en,12/9/1995,4300000.0,6.5,Four Rooms,4301331.0,4000000.0,Comedy|Fantasy,Salma Hayek,English,USA,1995.0,R,6.7,0,http://www.imdb.com/title/tt0113101/?ref_=fn_t...
1,9598,Babe,tt0112431,en,7/18/1995,254134910.0,6.0,Babe,66600000.0,30000000.0,Comedy|Drama|Family,Miriam Margolyes,English,Australia,1995.0,G,6.8,5000,http://www.imdb.com/title/tt0112431/?ref_=fn_t...
2,9603,Clueless,tt0112697,en,7/19/1995,0.0,6.9,Clueless,56631572.0,12000000.0,Comedy|Romance,Donald Faison,English,USA,1995.0,PG-13,6.8,0,http://www.imdb.com/title/tt0112697/?ref_=fn_t...
3,9312,Mortal Kombat,tt0113855,en,8/18/1995,122195920.0,5.4,Mortal Kombat,70360285.0,18000000.0,Action|Adventure|Fantasy|Sci-Fi|Thriller,Christopher Lambert,English,USA,1995.0,PG-13,5.8,0,http://www.imdb.com/title/tt0113855/?ref_=fn_t...
4,577,To Die For,tt0114681,en,5/20/1995,21284514.0,6.7,To Die For,21200000.0,20000000.0,Comedy|Crime|Drama,Kurtwood Smith,English,USA,1995.0,R,6.8,1000,http://www.imdb.com/title/tt0114681/?ref_=fn_t...
5,807,Se7en,tt0114369,en,9/22/1995,327311859.0,8.1,Se7en,100125340.0,33000000.0,Crime|Drama|Mystery|Thriller,Morgan Freeman,English,USA,1995.0,R,8.6,39000,http://www.imdb.com/title/tt0114369/?ref_=fn_t...
6,10530,Pocahontas,tt0114148,en,6/14/1995,346079773.0,6.7,Pocahontas,141600000.0,55000000.0,Adventure|Animation|Drama|Family|History|Music...,Christian Bale,English,USA,1995.0,G,6.6,0,http://www.imdb.com/title/tt0114148/?ref_=fn_t...
7,629,The Usual Suspects,tt0114814,en,7/19/1995,23341568.0,8.1,The Usual Suspects,23272306.0,6000000.0,Crime|Drama|Mystery|Thriller,Kevin Spacey,English,USA,1995.0,R,8.6,28000,http://www.imdb.com/title/tt0114814/?ref_=fn_t...
8,11359,The Indian in the Cupboard,tt0113419,en,7/14/1995,0.0,5.9,The Indian in the Cupboard,35617599.0,45000000.0,Drama|Family|Fantasy,Steve Coogan,English,USA,1995.0,PG,5.9,0,http://www.imdb.com/title/tt0113419/?ref_=fn_t...
9,10634,Friday,tt0113118,en,4/26/1995,28215918.0,7.0,Friday,27900000.0,3500000.0,Comedy|Drama,Nia Long,English,USA,1995.0,R,7.3,8000,http://www.imdb.com/title/tt0113118/?ref_=fn_t...


In [10]:
# Summaries
# Genre counts
genres = df[['genres', 'id']].groupby(['genres']).count()
genres = genres.rename(columns={'id': 'count'})

# Gross Revenue by Year
yearly_revenue = df[['title_year', 'revenue']].groupby(['title_year']).sum()

# Average ratings by revenue/budget bins
less_mil = df['revenue'] <= 1000000
mil1 = ((df['revenue'] > 1000000) & (df['revenue'] <= 25000000))
mil2 = ((df['revenue'] > 25000000) & (df['revenue'] <= 50000000))
mil3 = ((df['revenue'] > 50000000) & (df['revenue'] <= 75000000))
mil4 = ((df['revenue'] > 75000000) & (df['revenue'] <= 100000000))
mil5 = df['revenue'] > 100000000

df.loc[less_mil, "rev_cat"] = 'Less than a million'
df.loc[mil1, "rev_cat"] = '1 to 25 million'
df.loc[mil2, "rev_cat"] = '25 to 50 million'
df.loc[mil3, "rev_cat"] = '50 - 75 million'
df.loc[mil4, "rev_cat"] = '75 - 100 million'
df.loc[mil5, "rev_cat"] = '100+ million'
df.head()

ratings = df[['rev_cat', 'imdb_score']].groupby(['rev_cat']).mean()